In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

In [7]:
data = pd.read_csv("feature data/scaled.csv")
data.loc[data["0"] == 10, "0"] = 1
data

,0,Left Eye Width,Left Eye Height,Right Eye Width,Right Eye Height,Mouth Width,Mouth Height
0,0.0,0.709524,0.475776,0.691477,0.454649,0.428024,0.389152
1,0.0,0.589949,0.472364,0.525904,0.476794,0.329413,0.406707
2,0.0,0.560594,0.368381,0.437098,0.407039,0.267960,0.389357
3,0.0,0.499240,0.333411,0.592178,0.352945,0.413974,0.338359
4,0.0,0.525734,0.319817,0.469527,0.289953,0.275131,0.362585
...,...,...,...,...,...,...,...
479,1.0,0.364898,0.555501,0.665890,0.367276,0.442842,0.461740
480,1.0,0.310230,0.221509,0.613845,0.302325,0.520804,0.437043
481,1.0,0.304316,0.211383,0.653861,0.160434,0.576461,0.437043
482,1.0,0.253852,0.310932,0.622044,0.282020,0.379948,0.438876


In [8]:
X = data.iloc[:, [i for i in range(1, 7)]]
Y = data.iloc[:, 0]
X

,Left Eye Width,Left Eye Height,Right Eye Width,Right Eye Height,Mouth Width,Mouth Height
0,0.709524,0.475776,0.691477,0.454649,0.428024,0.389152
1,0.589949,0.472364,0.525904,0.476794,0.329413,0.406707
2,0.560594,0.368381,0.437098,0.407039,0.267960,0.389357
3,0.499240,0.333411,0.592178,0.352945,0.413974,0.338359
4,0.525734,0.319817,0.469527,0.289953,0.275131,0.362585
...,...,...,...,...,...,...
479,0.364898,0.555501,0.665890,0.367276,0.442842,0.461740
480,0.310230,0.221509,0.613845,0.302325,0.520804,0.437043
481,0.304316,0.211383,0.653861,0.160434,0.576461,0.437043
482,0.253852,0.310932,0.622044,0.282020,0.379948,0.438876


In [9]:
from sklearn.model_selection import train_test_split
X_Train, X_Test, Y_Train, Y_Test = train_test_split(X, Y, test_size = 0.20, shuffle=False)

In [92]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(10, input_shape=[6]),
    tf.keras.layers.Dense(100, activation="relu"),
    tf.keras.layers.Dense(2, activation="softmax")
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_69 (Dense)             (None, 10)                70        
_________________________________________________________________
dense_70 (Dense)             (None, 100)               1100      
_________________________________________________________________
dense_71 (Dense)             (None, 2)                 202       
Total params: 1,372
Trainable params: 1,372
Non-trainable params: 0
_________________________________________________________________


In [93]:
model.fit(X_Train, Y_Train, epochs=1000)

Epoch 1/1000
13/13 [==============================] - 1s 2ms/step - loss: 0.6930 - accuracy: 0.5430
Epoch 2/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.6907 - accuracy: 0.5320
Epoch 3/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.6877 - accuracy: 0.5681
Epoch 4/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.6852 - accuracy: 0.5304
Epoch 5/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.6814 - accuracy: 0.5424
Epoch 6/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.6808 - accuracy: 0.5964
Epoch 7/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.6832 - accuracy: 0.5795
Epoch 8/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.6791 - accuracy: 0.6140
Epoch 9/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.6741 - accuracy: 0.5949
Epoch 10/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.6748 - accuracy: 0.5922

13/13 [==============================] - 0s 2ms/step - loss: 0.5372 - accuracy: 0.7709
Epoch 83/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.5490 - accuracy: 0.7298
Epoch 84/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.5344 - accuracy: 0.7578
Epoch 85/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.5557 - accuracy: 0.7228
Epoch 86/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.5411 - accuracy: 0.7377
Epoch 87/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.5457 - accuracy: 0.7158
Epoch 88/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.5131 - accuracy: 0.7473
Epoch 89/1000
13/13 [==============================] - 0s 1ms/step - loss: 0.5309 - accuracy: 0.7409
Epoch 90/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.5590 - accuracy: 0.7331
Epoch 91/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.5448 - accuracy: 0.7335
Epoc

13/13 [==============================] - 0s 1ms/step - loss: 0.4794 - accuracy: 0.7860
Epoch 163/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.4872 - accuracy: 0.7757
Epoch 164/1000
13/13 [==============================] - 0s 1ms/step - loss: 0.4618 - accuracy: 0.8204
Epoch 165/1000
13/13 [==============================] - 0s 1ms/step - loss: 0.5216 - accuracy: 0.7586
Epoch 166/1000
13/13 [==============================] - 0s 1ms/step - loss: 0.4925 - accuracy: 0.7822
Epoch 167/1000
13/13 [==============================] - 0s 1ms/step - loss: 0.4676 - accuracy: 0.8100
Epoch 168/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.4698 - accuracy: 0.7846
Epoch 169/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.4617 - accuracy: 0.7971
Epoch 170/1000
13/13 [==============================] - 0s 1ms/step - loss: 0.4969 - accuracy: 0.7537
Epoch 171/1000
13/13 [==============================] - 0s 1ms/step - loss: 0.4529 - accuracy: 0.

13/13 [==============================] - 0s 1ms/step - loss: 0.4411 - accuracy: 0.7976
Epoch 243/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.4465 - accuracy: 0.8033
Epoch 244/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.4427 - accuracy: 0.8079
Epoch 245/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.4608 - accuracy: 0.8077
Epoch 246/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.4410 - accuracy: 0.8062
Epoch 247/1000
13/13 [==============================] - 0s 1ms/step - loss: 0.4911 - accuracy: 0.7677
Epoch 248/1000
13/13 [==============================] - 0s 1ms/step - loss: 0.4365 - accuracy: 0.8284
Epoch 249/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.4284 - accuracy: 0.8083
Epoch 250/1000
13/13 [==============================] - 0s 1ms/step - loss: 0.4534 - accuracy: 0.7912
Epoch 251/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.4447 - accuracy: 0.

13/13 [==============================] - 0s 2ms/step - loss: 0.4067 - accuracy: 0.8203
Epoch 323/1000
13/13 [==============================] - 0s 1ms/step - loss: 0.4290 - accuracy: 0.8099
Epoch 324/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.3941 - accuracy: 0.8146
Epoch 325/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.3893 - accuracy: 0.8295
Epoch 326/1000
13/13 [==============================] - 0s 1ms/step - loss: 0.3750 - accuracy: 0.8214
Epoch 327/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.3975 - accuracy: 0.8198
Epoch 328/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.3943 - accuracy: 0.8272
Epoch 329/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.4255 - accuracy: 0.7876
Epoch 330/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.4166 - accuracy: 0.7987
Epoch 331/1000
13/13 [==============================] - 0s 1ms/step - loss: 0.3989 - accuracy: 0.

...

13/13 [==============================] - 0s 1ms/step - loss: 0.2896 - accuracy: 0.8707
Epoch 643/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.2868 - accuracy: 0.8799
Epoch 644/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.2762 - accuracy: 0.8684
Epoch 645/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.2493 - accuracy: 0.8855
Epoch 646/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.2939 - accuracy: 0.8618
Epoch 647/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.2773 - accuracy: 0.8864
Epoch 648/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.2538 - accuracy: 0.9047
Epoch 649/1000
13/13 [==============================] - 0s 1ms/step - loss: 0.2695 - accuracy: 0.8776
Epoch 650/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.2777 - accuracy: 0.8815
Epoch 651/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.2615 - accuracy: 0.

13/13 [==============================] - 0s 2ms/step - loss: 0.2890 - accuracy: 0.8645
Epoch 723/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.2577 - accuracy: 0.8905
Epoch 724/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.2688 - accuracy: 0.8797
Epoch 725/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.2521 - accuracy: 0.8887
Epoch 726/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.2567 - accuracy: 0.8963
Epoch 727/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.2662 - accuracy: 0.8835
Epoch 728/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.2679 - accuracy: 0.8861
Epoch 729/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.2588 - accuracy: 0.8770
Epoch 730/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.2934 - accuracy: 0.8801
Epoch 731/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.2556 - accuracy: 0.

13/13 [==============================] - 0s 2ms/step - loss: 0.2200 - accuracy: 0.9113
Epoch 803/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.2417 - accuracy: 0.8987
Epoch 804/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.2612 - accuracy: 0.9009
Epoch 805/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.2373 - accuracy: 0.9119
Epoch 806/1000
13/13 [==============================] - 0s 1ms/step - loss: 0.2483 - accuracy: 0.8893
Epoch 807/1000
13/13 [==============================] - 0s 1ms/step - loss: 0.2339 - accuracy: 0.9195
Epoch 808/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.2492 - accuracy: 0.9030
Epoch 809/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.2738 - accuracy: 0.8691
Epoch 810/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.2498 - accuracy: 0.8978
Epoch 811/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.2512 - accuracy: 0.

13/13 [==============================] - 0s 2ms/step - loss: 0.2343 - accuracy: 0.8822
Epoch 883/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.2329 - accuracy: 0.8975
Epoch 884/1000
13/13 [==============================] - 0s 1ms/step - loss: 0.2774 - accuracy: 0.8842
Epoch 885/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.2349 - accuracy: 0.8981
Epoch 886/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.2321 - accuracy: 0.9044
Epoch 887/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.2088 - accuracy: 0.9010
Epoch 888/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.2458 - accuracy: 0.8975
Epoch 889/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.2322 - accuracy: 0.8927
Epoch 890/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.2355 - accuracy: 0.9028
Epoch 891/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.2352 - accuracy: 0.

13/13 [==============================] - 0s 2ms/step - loss: 0.2313 - accuracy: 0.9193
Epoch 963/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.2083 - accuracy: 0.9163
Epoch 964/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.2018 - accuracy: 0.9142
Epoch 965/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.2329 - accuracy: 0.9112
Epoch 966/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.1985 - accuracy: 0.9284
Epoch 967/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.2065 - accuracy: 0.8999
Epoch 968/1000
13/13 [==============================] - 0s 1ms/step - loss: 0.2235 - accuracy: 0.8929
Epoch 969/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.2411 - accuracy: 0.9042
Epoch 970/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.2211 - accuracy: 0.9122
Epoch 971/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.2087 - accuracy: 0.

In [94]:
model.evaluate(X_Test, Y_Test)

4/4 [==============================] - 0s 2ms/step - loss: 0.5276 - accuracy: 0.8041


[0.5275923609733582, 0.8041236996650696]